In [1]:
## database imports
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase

## data processor import
from processor.processor import Processor as p

## risk and return class imports
from returns.products import Products as returns_products_class
from risk.quarterly_risk import QuarterlyRisk as risk_class
from returns.quarterly_returns import QuarterlyReturns as returns_class

## strategy filters
from strategy_filters.quarterly_filter import QuarterlyFilter as quarterly_filter_class

## backtester import
from backtester.quarterly_backtester import QuarterlyBacktester as b

## dividend ticker transform
from strategy.dividends import Dividends as dividends_class

## additional imports
from datetime import datetime
import pandas as pd
from tqdm import tqdm

In [2]:
start_date = datetime(2023,1,1)
end_date = datetime.now()

In [3]:
market = Market()
sec = SEC()
fin_db = ADatabase("financial")
earnings_db = ADatabase("earnings")
dividends_db = ADatabase("dividends")
current_db = ADatabase("all_financial")

In [4]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [5]:
tyields = returns_products_class.tyields()
bench_returns = returns_products_class.spy_bench()

In [6]:
new_prices = []
market.connect()
sec.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        financials = sec.retrieve_filing_data(cik)
        ticker_sim = market.retrieve_ticker_prices("prices",ticker)
        ticker_sim = returns_class.returns(ticker_sim,financials)
        completed = risk_class.risk(ticker_sim,bench_returns)
        new_prices.append(completed)
    except:
        continue
sec.disconnect()
market.disconnect()
price_returns = pd.concat(new_prices)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [03:05<00:00,  2.64it/s]


In [7]:
fin_db.connect()
financial_simulation = fin_db.retrieve("predictions")
fin_db.disconnect()

In [8]:
earnings_db.connect()
earnings_simulation = earnings_db.retrieve("predictions")
earnings_db.disconnect()
earnings_simulation.rename(columns={"prediction":"earnings_prediction"},inplace=True)

In [9]:
dividends_db.connect()
dividends_simulation = dividends_db.retrieve("predictions")
dividends_db.disconnect()
dividends_simulation.rename(columns={"prediction":"dividends_prediction"},inplace=True)

In [10]:
sim = price_returns.merge(tyields[["year","quarter","quarterly_yield"]],on=["year","quarter"],how="left") \
                    .merge(financial_simulation[["year","quarter","ticker","financial_prediction"]],on=["year","quarter","ticker"],how="left") \
                    .merge(earnings_simulation[["year","quarter","ticker","earnings_prediction"]],on=["year","quarter","ticker"],how="left") \
                    .merge(dividends_simulation[["year","quarter","ticker","dividends_prediction"]],on=["year","quarter","ticker"],how="left")
sim = sim.dropna().groupby(["year","quarter","ticker"]).mean().reset_index()

In [11]:
ranks = sim.merge(sp500[["ticker","GICS Sector"]],how="left").groupby(["year","quarter","GICS Sector"]).mean().reset_index().sort_values(["year","quarter","quarterly_return"],ascending=False).groupby(["year","quarter"]).first().reset_index().rename(columns={"GICS Sector":"top_sector"})[["year","quarter","top_sector"]]
ranks["year"] = ranks["year"] + 1

In [12]:
sim.sort_values(["year","quarter"],inplace=True)

In [13]:
dividend_tickers = dividends_class.dividend_tickers(price_returns)

In [14]:
values = [True,False]
ceilings = [True,False]
parameters = []
positions = 10
strats = ["dividends","earnings","sector","financials"]
for strat in strats:
    for value in values:
        for ceiling in ceilings:
            parameter = {"value":value
                         ,"ceiling":ceiling
                        ,"strat":strat
                        }
            parameters.append(parameter)

In [15]:
len(parameters)

16

In [16]:
parameter = parameters[0]

In [17]:
current_db.connect()
current_db.drop("trades")
for parameter in tqdm(parameters):
    simulation = sim.copy()
    strat = parameter["strat"]
    simulation = returns_class.returns_backtest(simulation,strat,dividend_tickers)
    simulation = quarterly_filter_class.strategy_filter(strat,simulation,sp500,ranks)
    b.backtest(simulation.copy(),positions,parameter,start_date,end_date,current_db)
current_db.disconnect()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 16.91it/s]


In [18]:
simulation.columns

Index(['year', 'quarter', 'ticker', 'quarter_start', 'quarter_end',
       'return_end', 'commonstockdividendspersharecashpaid',
       'quarterly_return', 'bench_quarterly_return', 'quarterly_variance',
       'market_quarterly_cov', 'quarterly_beta', 'quarterly_yield',
       'financial_prediction', 'earnings_prediction', 'dividends_prediction',
       'market_quarterly_return', 'projected_quarterly_return',
       'quarterly_delta', 'quarterly_delta_sign', 'quarterly_rrr'],
      dtype='object')